# Dissecting the transcriptional effects of somatic mutations in human cancers
### *** Andrea Garofoli***

## Original Full Project's Abstract
*Large-scale sequencing studies in human cancers have uncovered hundreds of thousands of novel somatic mutations, only some of which are likely to confer significant advantage to cancer cells. As exhaustive functional characterization of each novel mutation is unfeasible, distinguishing the ‘driver’/targetable mutations from the ‘passenger’ mutations presents significant clinical challenges. With the major highly recurrent mutations already characterized as oncogenic, I hypothesize that the pathogenicity and/or actionability of uncharacterized mutations could be predicted based on whether these mutations lead to transcriptomic changes that resemble those of known activating mutations. Here I aim 1) to generate gene signatures for known activating mutations in oncogenes and 2) to infer the pathogenicity of uncharacterized mutations and to functionally benchmark the predictions. To achieve these aims, I will first define a gene signature for each known activating mutation and validate the gene signatures in an independent cohort. I will then predict the pathogenicity of the uncharacterized mutations and benchmark the predictions by testing the oncogenic potential of a subset of mutations and their response to targeted agents using in vitro and/or in vivo models. I anticipate that the results will serve as an extensive and valuable resource and will provide a computational framework to predict the significance of novel mutations for the scientific and clinical communities.*


## Python Project's Introduction

This python project's aim is to perform the first phase of this study.
The data I've worked comes from the **METABRIC** cohort (*Nature 2012 & Nat. Commun. 2016*) and was previously collected and reviewed by my supervisor. 
These first tests were performed on the *PIK3CA* mutation. The many *in vivo* and *in vitro* information that could be easily found in the literature made it easy for us to see if the approach actually works.

There are 4 main phases:
+ Sampe Selection
+ Feature Selection
+ Model Building and Validation
+ Infer mutations with similiar patterns


## Input Files

+	**Normalized Gene Expression** tables for the *Training* and the *Validation* sets. 
+ + *TrainingTable*’s size: 48803 rows (genes) and 960 columns (samples).
+ +	*ValidationTable*’s size: 48803 rows and 949 columns.

+ One table with **mutations** related information (such as chromosome, locus, gene name, codon, etc.) for each sample in both datasets.

+ One table with **Copy Number Alterations** related information for each sample in both datasets.

+ One table with **Histopathological Features** (such as age, er and her2 status, tumor size, etc.) specifics for each sample in both datasets.

All the files are the result of an internal pipeline, so they have a fixed structure and fixed names, this script was built with that in mind.
The Gene expression tables are tab separated and stored in .txt files, the other tables are comma separated and stored in .csv files.

## Output
The script can return the following files and information:
+ Two tables with the data subsets that will be used to build the machine learning model (more info in the next paragraph)
+ Two tables with the data subsets that will be used to obtain the list of oncogenic *PIK3CA* mutations
+ The list of genes that have the most different gene expression between the breast cancer subtype we are studying
+ The list of uncharacterized and potentially oncogenic mutations in *PIK3CA*

### Sample Selection 

Here I classify the samples in two groups, the **Mutant** (samples with a mutation in *PIK3CA*) and the **Control** one (so the samples with a wt *PIK3CA*). Furthermore, I will identify and remove 2 subgroups from the Mutant sample lists by finding samples with potentially confounding mutations and histopathological features (for examples, the activation of the *PI3K* pathway may be the effect of a *PIK3CA* mutation but also the result of a mutation in *PTEN*, *AKT1* and many others) and samples with a *PIK3CA* mutation in different places than the *H1047R*, *E542K* and *E545K* hotspots. In a later phase the classification model will be used on this latter subgroup to infer potentially pathogenic/actionable mutations.

The four types of samples are associated to four different colors:
+ ***Red*** = **Mutant** samples
+ ***Grey*** = **Control** samples
+ ***Purple*** = Samples removed due to the presence of potentially confounding mutations or histopoathological features.
+ ***Blue*** = Samples with *PIK3CA* mutations in different places than the 3 hotspots

Comments on the implementation can be found within the code blocks.


In [1]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
import glob, os
import re    


####
##   ChooseFolder
## This function was made to made it possible for the user to select
## the path for the folder with the Input files.
## The function will also check for the presence of eventual output files
## already made in a previous run of the script (which has to be removed manually
## by the user).
##
####


def ChooseFolder():
    root = tk.Tk()
    root.withdraw()
    root.update()
    print("Select the Working Folder")
    dirpath = filedialog.askdirectory(initialdir=os.getcwd(),
                                       title="Select the Working Folder")
    root.update()
    root.destroy()
    
    
    if not len(glob.glob(dirpath+"/*.txt"))==2 and not len(glob.glob(filename+"/*.csv"))==3:
        messagebox.showinfo("Visualizer error", "Wrong number of files, maybe you didn't choose the right directory?")
        sys.exit("Error, wrong file or directory")
    if not len(glob.glob(dirpath+"/temp/*"))==0:
        messagebox.showinfo("Visualizer error", "Folder not empty, cannot be used for the temp files")
        sys.exit("Error, wrong file or directory")
    if not len(glob.glob(dirpath+"/out/*"))==0:
        messagebox.showinfo("Visualizer error", "Folder not empty, cannot be used for the output")
        sys.exit("Error, wrong file or directory")        
    return(dirpath)


####
##   FiltList
## This function will read the input files (see the documentation for more info about them)
## and make a list of the samples that need to be filtered
## from the datase.
## The function will check for the presence of said input files.
##
####

def FiltList(tabfo):
    print(tabfo)            

    try:
        muta = open(tabfo +'/mut.csv',"r").read().split('\n')
    except FileNotFoundError:
        sys.exit("Mutation file cannot be found, was it renamed or moved?")


    print("Preparing lists")


    try:
        m = open(tabfo +'/pheno_r_file.csv',"r").read().split('\n')
    except FileNotFoundError:
        sys.exit("Phenotype file cannot be found, was it renamed or moved?")
    
    
    phen=[]

# Histopathological features used to filter the samples: ER HER2 status and histology    
    
    for n in range(len(m)-1):
        x=m[n].split(",")
        for i in range(len(x)):
            if x[3]=="POS" and x[4]=="NEG":
                if x[10]=="Ductal/NST" :
                    phen.append(x[1])
    
    phen=sorted(set(phen))

# This is the list of the potentially confounding mutations

    lismut=["AKT1", "AKT2", "AKT3", "CDKN1A", "CDKN1B", "ERBB2", "ERBB3", "ERBB4", "FGFR1", "FGFR2", "FGFR3",
#            "FGFR4", "GRB2", "HIF1A", "HRAS", "IGF1R", "IRS1", "KIT", "MTOR", "NRAS", "PDGFRA", "PDGFRB", 
#            "PIK3CB", "PIK3R1", "PRKCA", "PRKCB", "PRKCG", "PTEN", "RICTOR", "RPS6KB1", "RPTOR", "SOS1", 
#            "TSC1", "TSC2"]
#    lismut=["AKT1", "AKT2", "AKT3", "CDKN1A", "CDKN1B", "PIK3CA", "ERBB3", "ERBB4", "FGFR1", "FGFR2", "FGFR3",
            "FGFR4", "GRB2", "HIF1A", "HRAS", "IGF1R", "IRS1", "KIT", "MTOR", "NRAS", "PDGFRA", "PDGFRB", 
            "PIK3CB", "PIK3R1", "PRKCA", "PRKCB", "PRKCG", "PTEN", "RICTOR", "RPS6KB1", "RPTOR", "SOS1", 
            "TSC1", "TSC2"]

    lmu=[]

    for n in range (1, len(muta)-1):
        x = muta[n].split(',')
        x[9]=x[9]
        if x[9] in lismut  :
            lmu.append(x[1])

    lmu=sorted(set(lmu))
    
    Hmu=[]
    blmu=[]

# Here the script separates the PIK3CA mutants into the Red and Blue subsets    
    
    for n in range (1, len(muta)-1):
        x = muta[n].split(',')
        x[9]=x[9]
        if x[9]=="PIK3CA":
#        if x[9]=="ERBB2":            
            if x[12] == "1047" or x[12] == "545" or x[12] == "542":
#            if x[12] == "755" or x[12] == "777" or x[12] == "769":
                Hmu.append(x[1])
            else:
                blmu.append(x[1])

    Hmu=sorted(set(Hmu))

    blmu=sorted(set(blmu))


    try:
        p = open(tabfo +"/CN.csv","r").read().split('\n')
    except FileNotFoundError:
        sys.exit("CopyNumber file cannot be found, was it renamed or moved?")
        
    li=[]
    lik=[]

    pa=p[0].split(',')

    for i in range(1, len(p)-1):
        x=p[i].split(',')
        if x[7] in lismut:
            for n in range(7,len(x)):
                if x[n]=="-2" or x[n]=="2":
                    li.append(pa[n])
                else: lik.append(pa[n])

    lif=sorted(set(li))
    
    return(phen, lmu, Hmu, blmu, lif)


####
##   Filt
## This function will create the Mutation and Validation dataset
## (see the documentation for more info about them).
## The function will also print on screen some additional info about the
## filtering phase such as the number of samples found in for each criteria
## (for example ER+ and ER-) and the state of progress of each step
## (in the form of "processed rows / total rows").
##
####


    
def Filt(er, mu, hm, bl, cn, file, tabfo):               

    try:
        di = open(file,"r").read().split('\n')
    except FileNotFoundError:
        sys.exit("Gene expression file cannot be found, was it renamed or moved?")
        
    fname=re.split(r'[\.-\/]',file)[-2]
    
    print("Preparing the " + fname + " dataset")
    
    der=[""]*len(di)

    paz=di[0].split(',')


    c=[0]*len(paz)  

    for i in range(1, len(paz)):
        if  paz[i] not in er:
            c[i]= 1


    print("Number of samples for each ER status:\n ER + ",c.count(0)-1,"ER - ", c.count(1))


    print("Samples processed")

    for n in range(len(di)-1):
        if n % 10000 == 0 : print(str(n) + "/" + str(len(di)))    
        x=di[n].split(",")
        der[n]=x[0]
        for i in range(1, len(c)):
            if c[i] == 0 :
                der[n]=der[n]+ "," + x[i] 


    di= der
    der=[""]*len(di)

    paz=di[0].split(',')


    c=[0]*len(paz)


    for i in range(1, len(paz)):
        if  paz[i] in hm :
            c[i]= 2
        elif paz[i] in mu or paz[i] in cn :
            c[i]= 1

            
    print("Number of samples for each subset:\n Gray ", c.count(0)-1,"Purple ", c.count(1),"Red ", c.count(2))

    print("Samples processed")
    
    for n in range(len(di)-1):
        if n % 10000 == 0 : print(str(n) + "/" + str(len(di)))    
        x=di[n].split(",")
        der[n]=x[0]
        for i in range(1, len(c)):
            if c[i] != 1 :
                der[n]=der[n]+ "," + x[i]



    di= der
    der=[""]*len(di)

    paz=di[0].split(',')

    c=[0]*len(paz) 


    for i in range(len(paz)):
        if  paz[i] in bl and paz[i] not in hm:
            c[i]= 1


    print("Number of samples for each subset:\n Gray and Red ", c.count(0)-1,"Blue ", c.count(1))



    print("Samples processed")

    for n in range(len(di)-1):
        if n % 10000 == 0 : print(str(n) + "/" + str(len(di)))    
        x=di[n].split(",")
        der[n]=x[0]
        for i in range( len(c)):
            if c[i] == 0 and i!=0 :
                der[n]=der[n]+ "," + x[i]
            elif c[i] == 1 or i==0:
                if i == 0:
                    with open(tabfo + '/out/data_blu_'+ fname+'.csv', 'a') as out:
                        out.write(x[i])
                else:
                    with open(tabfo + '/out/data_blu_'+ fname+'.csv', 'a') as out:
                        out.write("," + x[i] )
        with open(tabfo + '/out/data_blu_'+ fname+'.csv', 'a') as out:
            out.write("\n")

    di= der
    der=[""]*len(di)

    paz=di[0].split(',')


    c=[1]*len(paz) 

    for i in range(1, len(paz)):
        if  paz[i] in hm:
            c[i]= 2


    print("Number of samples for each subset:\n Gray ",c.count(1)-1,"Red ", c.count(2))

            
    for n in range(len(di)):
        with open(tabfo + '/out/Vdataset'+ fname+'.csv', 'a') as out:
            out.write(di[n] + "\n")
            
    filelist = glob.glob(tabfo + "/temp/*.csv")
    for f in filelist:
        os.remove(f)            


    
    print(fname + " dataset complete")
    input("Press enter to continue")    
    
    return(c[1:len(c)])

####
##  SampleSelection
## This function simply performs the call to the other functions  twice,
## once for the Training dataset, once for the Validation one.
## Gene expression input files are chosen by the glob function.
##
####



def SampleSelection (tabfo):
    classes=[]
    for file in glob.glob(tabfo+"/*.txt"):

        phen, lmu, Hmu, blmu, lif = FiltList(tabfo)
        os.system('clear')  
        classes.append(Filt(phen, lmu, Hmu, blmu, lif, file, tabfo))

    return(classes)
 


    
tabfo=ChooseFolder()

print("The working folder is " + tabfo)

clasf= SampleSelection(tabfo)

    





Select the Working Folder
The working folder is /Users/andrea/Documents/BI/BI/B/PhD/PythonProject/PythonProject
/Users/andrea/Documents/BI/BI/B/PhD/PythonProject/PythonProject
Preparing lists
Preparing the Training dataset
Number of samples for each ER status:
 ER +  495 ER -  465
Samples processed
0/48805
10000/48805
20000/48805
30000/48805
40000/48805
Number of samples for each subset:
 Gray  164 Purple  152 Red  179
Samples processed
0/48805
10000/48805
20000/48805
30000/48805
40000/48805
Number of samples for each subset:
 Gray and Red  306 Blue  37
Samples processed
0/48805
10000/48805
20000/48805
30000/48805
40000/48805
Number of samples for each subset:
 Gray  127 Red  179
Training dataset complete
Press enter to continue
/Users/andrea/Documents/BI/BI/B/PhD/PythonProject/PythonProject
Preparing lists
Preparing the Validation dataset
Number of samples for each ER status:
 ER +  435 ER -  514
Samples processed
0/48805
10000/48805
20000/48805
30000/48805
40000/48805
Number of sampl

In [20]:
from pandas import *
import pandas as pd
from scipy.stats import ttest_ind

####
##  TopFeatures
## This function will perform the T-test and obtain the 
## mostly differentiate genes between the two subsets
## 
####


def TopFeatures(tabfo, ngen):
    print("Script is trying to obtain the top "+ str(ngen) +" features." )
    mu = pd.read_csv(tabfo + '/out/VdatasetTraining.csv', sep= ",", index_col=0)

    mu = mu.transpose()

    indWT = [i for i, x in enumerate(clasf[0]) if x == 1]
    indMU = [i-1 for i, x in enumerate(clasf[0]) if x == 2]

    tt=ttest_ind(mu.iloc[indWT], mu.iloc[indMU])


    mu = mu.transpose()

    mu = mu.assign(pvalue=pd.Series(tt[1]).values)
    #mu = mu.assign(tvalue=pd.Series(tt[0]).values)
    mu = mu.sort_values('pvalue') 
    
    mu.head(n=100).to_csv('GS.csv', index=True, header=True)
    
    mu.drop(mu.columns[[-1]], axis=1, inplace=True)
    mu=mu.head(n=ngen)

    return(mu, list(mu.index))


    
####
##  NumGenes
## This function lets the user decide the number of features to use 
## to build the model
## 
####

def NumGenes(prompt="Please enter how many features you want to use: "):

    while True:
        num_str = input(prompt).strip()
        if all(c in '0123456789' for c in num_str):
            break
        else:
            sys.exit("Please write an integer")
    return int(num_str)


ngen = NumGenes()
mu, glist = TopFeatures(tabfo, ngen)
#mu, glist = TopFeatures2(tabfo, 100)


Please enter how many features you want to use: 1000
Script is trying to obtain the top 1000 features.


### Feature Selection 

Feature selection is a critical step when dealing with machine learning. 
Using too few features will make the model work poorly, using too many features will make the model overfit to the training data.
The test dataset I am working on has almost 50000 different genes, definitely too many.
After testing many different approaches I've decided to perform a T-test between the gene expressions found respectively within the Red and the Gray samples. Rows are then reordered using the T test's outpur and only the top different expressed genes are chosen.


#### Optional: Testing the Machine Learning Methods

In order to choose the best Machine Learning approach I've tried different methods, using and adapting to my data a snippet of code found in the Scikit-Learn's documentation. This is not part of the pipeline but it was helpful to me to proceed with the next step.

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors
from sklearn import svm


####
##  TestML
## This function tries a number of different Machine Learning
## approaches and returns their overall accuracies by performing
## 10 fold Cross Validations.
## 
####


def TestML(mu, clasf):
    dataset_array = mu.values
    get=np.transpose(dataset_array)

    target=clasf[0]


    clf1 = LogisticRegression(penalty="l1", solver="liblinear", max_iter=10, C=1)
    clf2 = RandomForestClassifier(criterion="entropy", max_features="log2", n_estimators= 250)
    clf3 = GaussianNB()
    clf4 = SGDClassifier(alpha= 0.1, class_weight= "balanced", max_iter= 1000,  penalty= "elasticnet")
    clf5 = tree.DecisionTreeClassifier(criterion= 'entropy', max_depth= 5, max_features= 6, min_samples_leaf= 0.1, min_samples_split= 0.5, splitter= "best")
    clf6 = MLPClassifier(activation= "tanh", alpha= 0.001, hidden_layer_sizes= (50, 50, 100), solver='adam')
    clf7 = neighbors.KNeighborsClassifier(algorithm= 'auto', leaf_size= 1, n_neighbors= 10, p= 1, weights= 'uniform')
    clf8 = svm.SVC(kernel='linear')

    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), ('SGD', clf4), ('dt', clf5), 
                                        ('mlp', clf6), ('nn', clf7), ('svc', clf8)], voting='hard')
    print("Testing different Machine Learning Methods:\n")

    for clf, label in zip([clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf8, eclf], 
                          ['Logistic Regression', 'Random Forest', 'Naive Bayes', 'Ensemble', 
                           'Stochastic Gradient Descent', 'Decision Tree', 'Multilayer Perceptron','Nearest Neighbor', 'Support Vector Machines']):
        scores = cross_val_score(clf, get, target, cv=10, scoring='accuracy')


        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
        
TestML(mu, clasf)

Testing different Machine Learning Methods:

Accuracy: 0.82 (+/- 0.06) [Logistic Regression]
Accuracy: 0.78 (+/- 0.07) [Random Forest]
Accuracy: 0.75 (+/- 0.09) [Naive Bayes]
Accuracy: 0.80 (+/- 0.06) [Ensemble]
Accuracy: 0.64 (+/- 0.09) [Stochastic Gradient Descent]
Accuracy: 0.77 (+/- 0.09) [Decision Tree]
Accuracy: 0.80 (+/- 0.07) [Multilayer Perceptron]
Accuracy: 0.77 (+/- 0.06) [Nearest Neighbor]
Accuracy: 0.78 (+/- 0.07) [Support Vector Machines]


### Model Building and Validation

Using the subset of samples and genes selected in the previous steps now I build the actual classification model.
I've chosen to use the Logistic Regression approach since in the previous tests it has proven to be the one with the best accuracy.

In [22]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_selection import RFE


####
##  ModelBuild
## This function builds the model for the classification.
## It uses the Logistic Regression since it was the approach with the highest
## accuracy. It also performs a 10 fold Cross Validation and prints on screen
## the accuracies
## 
####

def ModelBuild(mu, clasf):
    print("The script is building the model")

    dataset_array = mu.values
    get=np.transpose(dataset_array)

    target=clasf[0]

    clf = LogisticRegression(penalty="l1", solver="liblinear", max_iter=10, C=1)

    print("The script is testing the model on the training data")

    wee=[0.70, 0.67, 0.65, 0.68, 0.58, 0.67, 0.60, 0.61, 0.68, 0.48, 0.74, 0.62, 0.53, 0.57, 0.55, 0.65, 0.63, 0.60, 0.67, 0.62, 0.69, 0.64, 0.80, 0.71, 0.65, 0.48, 0.75, 0.68, 0.61, 0.46, 0.66, 0.61, 0.50, 0.64, 0.62, 0.59, 0.55, 0.67, 0.55, 0.58, 0.70, 0.50, 0.59, 0.61, 0.61, 0.41, 0.57, 0.68, 0.61, 0.44, 0.50, 0.60, 0.68, 0.47, 0.61, 0.67, 0.60, 0.57, 0.70, 0.63, 0.53, 0.64, 0.45, 0.54, 0.48, 0.50, 0.66, 0.58, 0.60, 0.65, 0.58, 0.66, 0.58, 0.70, 0.57, 0.49, 0.58, 0.71, 0.54, 0.59, 0.52, 0.58, 0.53, 0.56, 0.58, 0.56, 0.61, 0.50, 0.55, 0.60, 0.57, 0.61, 0.58, 0.63, 0.50, 0.54, 0.60, 0.75, 0.68, 0.40, 0.37, 0.66, 0.76, 0.68, 0.66, 0.38, 0.60, 0.70, 0.72, 0.62, 0.62, 0.62, 0.74, 0.77, 0.47, 0.43, 0.49, 0.50, 0.37, 0.49, 0.48, 0.43, 0.61, 0.40, 0.71, 0.71, 0.77, 0.59, 0.58, 0.56, 0.55, 0.57, 0.40, 0.62, 0.49, 0.61, 0.58, 0.70, 0.36, 0.55, 0.56, 0.40, 0.56, 0.43, 0.47, 0.48, 0.60, 0.42, 0.65, 0.59, 0.83, 0.46, 0.72, 0.61, 0.56, 0.63, 0.57, 0.57, 0.58, 0.69, 0.67, 0.58, 0.76, 0.65, 0.62, 0.69, 0.56, 0.40, 0.53, 0.57, 0.56, 0.70, 0.65, 0.67, 0.83, 0.67, 0.79, 0.69, 0.68, 0.59, 0.52, 0.38, 0.51, 0.50, 0.51, 0.50, 0.61, 0.72, 0.78, 0.67, 0.60, 0.68, 0.60, 0.56, 0.58, 0.41, 0.55, 0.57, 0.58, 0.83, 0.55, 0.59, 0.55, 0.56, 0.49, 0.67, 0.59, 0.67, 0.53, 0.56, 0.63, 0.67, 0.72, 0.60, 0.74, 0.70, 0.63, 0.61, 0.53, 0.70, 0.60, 0.69, 0.73, 0.67, 0.54, 0.65, 0.63, 0.68, 0.78, 0.69, 0.83, 0.73, 0.63, 0.58, 0.79, 0.55, 0.71, 0.60, 0.71, 0.62, 0.75, 0.62, 0.53, 0.52, 0.67, 0.63, 0.73, 0.69, 0.71, 0.70, 0.67, 0.77, 0.68, 0.66, 0.83, 0.61, 0.62, 0.64, 0.84, 0.67, 0.62, 0.60, 0.63, 0.64, 0.67, 0.78, 0.74, 0.66, 0.81, 0.61, 0.54, 0.71, 0.53, 0.75, 0.65, 0.65, 0.64, 0.54, 0.56, 0.63, 0.65, 0.72, 0.60, 0.64, 0.42, 0.68, 0.76, 0.69, 0.85, 0.70, 0.71, 0.45, 0.55, 0.66, 0.58, 0.63, 0.57, 0.64, 0.50, 0.56, 0.64, 0.65, 0.51, 0.67, 0.67, 0.67]    
    
    clfe = clf.fit(get, target, sample_weight=wee)

    selector = SelectPercentile(f_classif, percentile=0.25)
    selector.fit(get, target)

    clfe = clf.fit(selector.transform(get), target, sample_weight=wee)
       
    targg=clfe.predict(selector.transform(get[:, :]))
    
    CM=confusion_matrix(target, targg)
    
    print("sensitivity (mut)\n",float(CM[1][1]/(CM[1][1]+CM[1][0])))   
    
    print("specificity (wt)\n", float(CM[0][0]/(CM[0][0]+CM[0][1])))
    
    print("overall accuracy\n", accuracy_score(target, targg))    
    
    cv=cross_val_score(clf, get, target, cv=10, scoring='accuracy')


    print("10 Fold Cross Validation Accuracies (Training Set):\n"+"\n".join(map(str,cv)))
    
    return(clfe)


np.random.seed(42)

clfe = ModelBuild(mu, clasf)

The script is building the model
The script is testing the model on the training data
sensitivity (mut)
 0.8938547486033519
specificity (wt)
 0.5354330708661418
overall accuracy
 0.745098039216
10 Fold Cross Validation Accuracies (Training Set):
0.870967741935
0.774193548387
0.709677419355
0.838709677419
0.677419354839
0.709677419355
0.774193548387
0.833333333333
0.9
0.793103448276


In [13]:
####
##  ValidationSet
## This function uses the model to classify the samples
## in the validation set and returns the accuracies
## from the 10 fold cross validation.
## 
####
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score



def ValidationSet(glist, clasf, clfe, mu):
    muv = pd.read_csv(tabfo + '/out/VdatasetValidation.csv', sep= ",", index_col=0)

    muv = muv.transpose()

    muv = muv[glist]

    valid_array = muv.values
    getv = valid_array

    targetv=clasf[1]

    target=clasf[0]
    
    dataset_array = mu.values
    get=np.transpose(dataset_array)

    selector = SelectPercentile(f_classif, percentile=0.25)
    selector.fit(get, target)
    
    targgv=clfe.predict(selector.transform(getv[:, :]))
    
    
    
    print("wt", clasf[1].count(1))            
    print("mut", clasf[1].count(2)) 
   

    CM=confusion_matrix(targetv, targgv)
    
    print("sensitivity (mut)\n",float(CM[1][1]/(CM[1][1]+CM[1][0])))   
    
    print("specificity (wt)\n", float(CM[0][0]/(CM[0][0]+CM[0][1])))
    
    print("overall accuracy\n", accuracy_score(targetv, targgv)) 

    cvv=cross_val_score(clfe, getv, targetv, cv=10, scoring='accuracy')

    print("10 Fold Cross Validation Accuracies (Validation Set):\n"+"\n".join(map(str,cvv)))

ValidationSet(glist, clasf, clfe, mu)

wt 107
mut 169
sensitivity (mut)
 0.9526627218934911
specificity (wt)
 0.4205607476635514
overall accuracy
 0.746376811594
10 Fold Cross Validation Accuracies (Validation Set):
0.785714285714
0.785714285714
0.857142857143
0.75
0.785714285714
0.714285714286
0.714285714286
0.777777777778
0.592592592593
0.769230769231


In [14]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from sklearn.datasets import make_classification


muv = pd.read_csv(tabfo + '/out/VdatasetValidation.csv', sep= ",", index_col=0)

muv = muv.transpose()

muv = muv[glist]

valid_array = muv.values
getv = valid_array
targetv=clasf[1]
for i in range(len(clasf[1])):
    if clasf[1][i]==1: targetv[i]=0
    elif clasf[1][i]==2: targetv[i]=1

#targetv=clasf[1]

y_predict_probabilities = clfe.predict_proba(getv)[:,1]

fpr, tpr, _ = roc_curve(targetv, y_predict_probabilities)

roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Validation set')
plt.legend(loc="lower right")
plt.show()


muv = pd.read_csv(tabfo + '/out/VdatasetTraining.csv', sep= ",", index_col=0)

muv = muv.transpose()

muv = muv[glist]

valid_array = muv.values
getv = valid_array

targetv2=[0]*len(clasf[0])

for i in range(len(clasf[1])):
    if clasf[0][i]==1: targetv2[i]=0
    elif clasf[0][i]==2: targetv2[i]=1

print(targetv2)
#targetv=clasf[1]

selector = SelectPercentile(f_classif, percentile=10)
selector.fit(getv, target)


y_predict_probabilities = clfe.predict_proba(selector.transform(getv))[:,1]

fpr, tpr, _ = roc_curve(targetv2, y_predict_probabilities)

roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Training set')
plt.legend(loc="lower right")
plt.show()

ValueError: X has 400 features per sample; expecting 1

# Infer mutations with similiar patterns

In this last phase we try to get the list of *PIK3CA* mutations with a gene expression pattern similiar to the ones in the hotspots used to define the "red" subset of samples. The script uses the logistic regression model previously made to infer these samples and performs a simple parsing of the mutation table to obtain the loci of said *PIK3CA* mutations

In [15]:

####
##  FindBlue
## This function uses the model to classify the samples
## in the "blue" datasets from both the Training and the Validation sets
## then it uses the list of samples classified as "red" to
## obtain the list of PIK3CA mutations' loci, classified
## by the machine learning, similiar to the ones in the hotspots.
## 
####

def FindBlue(blc, glist, clfe, file): 
    blt = pd.read_csv(file, sep= ",", index_col=0)

    blt = blt.transpose()

    blt = blt[glist]

    bt_array = blt.values
    getbt = bt_array

    target=clasf[0]
    
    dataset_array = mu.values
    get=np.transpose(dataset_array)
    selector = SelectPercentile(f_classif, percentile=0.25)
    selector.fit(get, target)
    
    
    targbt=clfe.predict(selector.transform(getbt[:, :]))
    print(clfe.predict_proba(selector.transform(getbt[:, :])))

    print(targbt)

    btMU = [i-1 for i, x in enumerate(targbt) if x == 2]


    btn=list(blt.index[btMU])

    print(btn)
    
    try:
        muta = open(tabfo +'/mut.csv',"r").read().split('\n')
    except FileNotFoundError:
        sys.exit("Mutation file cannot be found, was it renamed or moved?")



    for n in range (1, len(muta)-1):
        x = muta[n].split(',')
        x[9]=x[9]
        #print(x[1])
        if x[9]=="PIK3CA" and x[1] in btn:
#        if x[9]=="ERBB2" and x[1] in btn:
            blc.append(x[12])

    return(blc)

blc=[]            

for file in glob.glob(tabfo+"/out/data_*.csv"):
    FindBlue(blc, glist, clfe, file)


print("List of mutations in the blue subset classified similiar to the hotspots ones:\n" + "\n".join(map(str,list(set(blc)))))

print("\n\nlista \n"+ "\n".join(sorted(blc)))

[[ 0.5778247   0.4221753 ]
 [ 0.36371013  0.63628987]
 [ 0.40407598  0.59592402]
 [ 0.40798311  0.59201689]
 [ 0.40080111  0.59919889]
 [ 0.40561857  0.59438143]
 [ 0.43707263  0.56292737]
 [ 0.35850195  0.64149805]
 [ 0.46536018  0.53463982]
 [ 0.37133816  0.62866184]
 [ 0.36443282  0.63556718]
 [ 0.38638195  0.61361805]
 [ 0.54351152  0.45648848]
 [ 0.3586899   0.6413101 ]
 [ 0.38205786  0.61794214]
 [ 0.45959468  0.54040532]
 [ 0.39921972  0.60078028]
 [ 0.42628434  0.57371566]
 [ 0.60855485  0.39144515]
 [ 0.49072547  0.50927453]
 [ 0.53295577  0.46704423]
 [ 0.35811809  0.64188191]
 [ 0.38625497  0.61374503]
 [ 0.53161949  0.46838051]
 [ 0.34985203  0.65014797]
 [ 0.45321639  0.54678361]
 [ 0.50811281  0.49188719]
 [ 0.43928026  0.56071974]
 [ 0.40004912  0.59995088]
 [ 0.43379565  0.56620435]
 [ 0.36864642  0.63135358]
 [ 0.4167974   0.5832026 ]
 [ 0.48396281  0.51603719]
 [ 0.388379    0.611621  ]
 [ 0.48869183  0.51130817]
 [ 0.45274957  0.54725043]
 [ 0.43147422  0.56852578]]
